In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Design - Best Practices

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fprompts%2Fintro_prompt_design.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | |
|-|-|-|
|Author(s) | [Polong Lin](https://github.com/polong-lin) | [Karl Weinmeister](https://github.com/kweinmeister)|

## Overview

This notebook covers the essentials of prompt engineering, including some best practices.

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview).

In this notebook, you learn best practices around prompt engineering -- how to design prompts to improve the quality of your responses. 

This notebook covers the following best practices for prompt engineering:

- Be concise
- Be specific and well-defined
- Ask one task at a time
- Turn generative tasks into classification tasks
- Improve response quality by including examples

## Getting Started

### Install Vertex AI SDK and other required packages


In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [4]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [3]:
PROJECT_ID = "qwiklabs-gcp-04-41dcbc3a9952"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
from vertexai.generative_models import GenerationConfig, GenerativeModel
import time

### Load model

In [5]:
model = GenerativeModel("gemini-1.5-flash")

In [6]:
import time

def call_gemini(prompt, generation_config=GenerationConfig(temperature=1.0)):
    wait_time = 1
    while True:
        try:
            response = model.generate_content(prompt, generation_config=generation_config).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

def send_message_gemini(model, prompt):    
    wait_time = 1
    while True:
        try:
            response = model.send_message(prompt).text
            return response
            break  # Exit the loop if successful
        except Exception as e:  # Replace with the actual exception type
            time.sleep(wait_time)
            wait_time *= 2  # Double the wait time

## Prompt engineering best practices

Prompt engineering is all about how to design your prompts so that the response is what you were indeed hoping to see.

The idea of using "unfancy" prompts is to minimize the noise in your prompt to reduce the possibility of the LLM misinterpreting the intent of the prompt. Below are a few guidelines on how to engineer "unfancy" prompts.

In this section, you'll cover the following best practices when engineering prompts:

* Be concise
* Be specific, and well-defined
* Ask one task at a time
* Improve response quality by including examples
* Turn generative tasks to classification tasks to improve safety

### Be concise

🛑 Not recommended. The prompt below is unnecessarily verbose.

In [7]:
prompt = "What do you think could be a good name for a flower shop that specializes in selling bouquets of dried flowers more than fresh flowers?"

print(call_gemini(prompt))

Here are some names for a flower shop specializing in dried flowers:

**Whimsical & Romantic:**

* Everlasting Blooms
* The Dried Flower Garden
* Bloom & Preserve
* Whispers of Wildflowers
* The Paper Petal
* Eternal Blooms
* Dried Dreams

**Modern & Minimalist:**

* Botanica
* The Bloomery
* Bloom & Co.
* The Dried Flower Studio
* Wild & Free
* Timeless Blooms
* The Petal Project

**Unique & Playful:**

* The Flower Apothecary
* The Dusty Bloom
* Bloom & Dust
* The Paper Garden
* The Petal Press
* The Dried Flower Collective
* Bloom Forever

**Consider these factors when choosing a name:**

* **Target Audience:** Who are you trying to reach?
* **Brand Personality:** What feeling do you want your shop to evoke?
* **Availability:** Check if the name is available as a domain name and for social media handles.
* **Memorability:**  Is the name easy to remember and pronounce?

**Bonus Tip:** You can also incorporate your city or location into the name for a more personalized feel. For examp

✅ Recommended. The prompt below is to the point and concise.

In [8]:
prompt = "Suggest a name for a flower shop that sells bouquets of dried flowers"

print(call_gemini(prompt))

## Flower Shop Names for Dried Bouquets:

**Short & Sweet:**

* Bloom & Dry
* Everlasting Bloom
* Dried & Delicate
* The Dusty Bouquet
* Everbloom

**Descriptive & Evocative:**

* The Dried Flower Studio
* Whispers of Wildflowers
* Botanical Treasures
* The Timeless Bloom
* Forever in Bloom

**Unique & Playful:**

* Petal & Paper
* Dusty Rose Co.
* The Flower Whisperer
* The Botanical Alchemist
* Sun Kissed Blooms

**Location Inspired:**

* [Your City] Dried Flowers
* [Your Neighborhood] Blooms
* The [Your Street Name] Florist

**Bonus:**

* Consider using a name that reflects your brand's aesthetic, whether it's rustic, modern, or romantic.
* Check if the name is available as a domain name and social media handle. 

**Pro Tip:**

* Try combining words from different categories to create a unique and memorable name. For example, "Everlasting Whispers" or "The Dusty Flower Studio." 



### Be specific, and well-defined

Suppose that you want to brainstorm creative ways to describe Earth.

🛑 The prompt below might be a bit too generic (which is certainly OK if you'd like to ask a generic question!)

In [9]:
prompt = "Tell me about Earth"

print(call_gemini(prompt))

## Earth: Our Home Planet

Earth is the third planet from the Sun and the only known planet to harbor life. It's a vibrant, dynamic world with a diverse range of ecosystems, from the icy poles to the scorching deserts, lush forests to vast oceans.

**Here are some key facts about Earth:**

* **Size and Shape:** Earth is an oblate spheroid, meaning it's slightly flattened at the poles and bulging at the equator. Its diameter is approximately 12,742 kilometers.
* **Atmosphere:** Our planet is surrounded by a protective atmosphere composed mostly of nitrogen (78%) and oxygen (21%). This atmosphere traps heat, creating the greenhouse effect and allowing for life to thrive.
* **Surface:** Earth's surface is divided into landmasses (continents) and oceans. About 71% of the Earth's surface is covered by water.
* **Geology:** The Earth's crust is divided into tectonic plates that constantly move and interact, causing earthquakes, volcanoes, and mountain formation.
* **Climate:** Earth experien

✅ Recommended. The prompt below is specific and well-defined.

In [10]:
prompt = "Generate a list of ways that makes Earth unique compared to other planets"

print(call_gemini(prompt))

## What Makes Earth Unique:

**1. Liquid Water:** Earth is the only known planet in our solar system with abundant liquid water on its surface. This is crucial for life as we know it.

**2. Plate Tectonics:** Earth's crust is broken into plates that move, causing earthquakes, volcanoes, and mountain ranges. This dynamic system helps regulate Earth's climate and creates diverse environments.

**3. Atmosphere:** Earth's atmosphere is a complex mixture of gases, primarily nitrogen and oxygen. This atmosphere provides protection from harmful solar radiation, regulates temperature, and allows for weather patterns.

**4. Magnetic Field:** Earth's molten core generates a powerful magnetic field that acts as a shield, deflecting charged particles from the Sun (solar wind) and protecting life from damaging radiation.

**5. Distance from the Sun:** Earth is located in the "Goldilocks zone" - the perfect distance from the Sun for liquid water to exist, making it habitable.

**6. Moon:** Earth's m

### Ask one task at a time

🛑 Not recommended. The prompt below has two parts to the question that could be asked separately.

In [11]:
prompt = "What's the best method of boiling water and why is the sky blue?"

print(call_gemini(prompt))

Here's the breakdown of the best water boiling method and the science behind the blue sky:

**Boiling Water**

* **Best Method: Using a kettle or pot on a stovetop.** This is the most efficient and reliable method.
* **Why:**
    * **Control:** You can adjust the heat and monitor the water's progress easily.
    * **Speed:** Stoves provide direct heat, bringing water to a boil quickly.
    * **Safety:** Most kettles and pots are designed for safe boiling and have safety features.

**Other Methods**

* **Microwave:** Can be fast but doesn't always distribute heat evenly, potentially leading to hot spots.
* **Electric Kettle:** Convenient and fast, but less versatile for other cooking tasks. 

**Why the Sky is Blue**

The blue color of the sky is a result of a phenomenon called **Rayleigh scattering**. Here's the simplified explanation:

1. **Sunlight is made up of all colors of the rainbow.** 
2. **Sunlight enters the atmosphere and interacts with tiny particles of air.** These particle

✅ Recommended. The prompts below asks one task a time.

In [12]:
prompt = "What's the best method of boiling water?"

print(call_gemini(prompt))

The "best" method of boiling water depends on your needs and resources. Here's a breakdown of the most common options:

**For Speed and Simplicity:**

* **Electric Kettle:** This is the fastest and easiest method. Simply fill the kettle, plug it in, and wait for it to boil.  
* **Microwave:**  This is quick and convenient for small amounts of water. Be careful, as some microwaves don't have a boil setting and the water can overheat and erupt when you try to pour it. 

**For Stovetop Boiling:**

* **Pot on the Stove:**  This is the classic method. Choose a pot appropriate for the amount of water you need and heat it on a stovetop burner.  

**Factors to Consider:**

* **Amount of Water:** For small quantities, a microwave or electric kettle is best. For larger amounts, a pot on the stove is more efficient.
* **Speed:** Electric kettles are the fastest, followed by microwaves and then pots on the stove.
* **Energy Efficiency:** Electric kettles are generally the most energy efficient, fo

In [14]:
prompt = "Why is the sky blue?"

print(call_gemini(prompt))

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's how it works:

* **Sunlight is a mixture of colors:** White sunlight is actually a combination of all the colors of the rainbow.
* **Scattering by molecules:** When sunlight enters Earth's atmosphere, it interacts with tiny gas molecules like nitrogen and oxygen. These molecules are much smaller than the wavelengths of visible light.
* **Rayleigh scattering:**  These small molecules scatter the shorter wavelengths of light (blue and violet) much more strongly than the longer wavelengths (red and orange). This is because shorter wavelengths interact more with the molecules.
* **More blue light reaches our eyes:** As a result, blue light is scattered in all directions, while the longer wavelengths pass through the atmosphere with less scattering. This scattered blue light is what we see as the blue sky.

**Why the sky isn't violet:** Even though violet light is scattered even more strongly than blue, our eyes

### Watch out for hallucinations

Although LLMs have been trained on a large amount of data, they can generate text containing statements not grounded in truth or reality; these responses from the LLM are often referred to as "hallucinations" due to their limited memorization capabilities. Note that simply prompting the LLM to provide a citation isn't a fix to this problem, as there are instances of LLMs providing false or inaccurate citations. Dealing with hallucinations is a fundamental challenge of LLMs and an ongoing research area, so it is important to be cognizant that LLMs may seem to give you confident, correct-sounding statements that are in fact incorrect.

Note that if you intend to use LLMs for the creative use cases, hallucinating could actually be quite useful.

Try the prompt like the one below repeatedly. We set the temperature to 1.0 so that it takes more risks in its choices. It's possible that it may provide an inaccurate, but confident answer.

In [15]:
generation_config = GenerationConfig(temperature=1.0)

prompt = "What day is it today?"

print(call_gemini(prompt, generation_config))

I do not have access to real-time information, including the current date. To know what day it is today, please check a calendar or your device's date settings. 



Since LLMs do not have access to real-time information without further integrations, you may have noticed it hallucinates what day it is today in some of the outputs.

### Using system instructions to guardrail the model from irrelevant responses

How can we attempt to reduce the chances of irrelevant responses and hallucinations?

One way is to provide the LLM with [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini#system-instructions).

Let's see how system instructions works and how you can use them to reduce hallucinations or irrelevant questions for a travel chatbot.

Suppose we ask a simple question about one of Italy's most famous tourist spots.

In [16]:
model_travel = GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=[
        "Hello! You are an AI chatbot for a travel web site.",
        "Your mission is to provide helpful queries for travelers.",
        "Remember that before you answer a question, you must check to see if it complies with your mission.",
        "If not, you can say, Sorry I can't answer that question.",
    ],
)

chat = model_travel.start_chat()

prompt = "What is the best place for sightseeing in Milan, Italy?"

print(send_message_gemini(chat, prompt))

Milan is a fantastic city for sightseeing! To give you the best recommendation, could you tell me a little more about what you're interested in? 

For example:

* **What kind of sights are you looking for?**  (Historical landmarks, art museums, modern architecture, etc.)
* **How much time do you have?** (A quick visit, a day trip, or longer?)
* **What is your budget?** (Free attractions, paid entry, etc.) 

Once I know a bit more about your preferences, I can give you some tailored suggestions for the best sightseeing spots in Milan. 



Now let us pretend to be a user asks the chatbot a question that is unrelated to travel.

In [17]:
prompt = "What's for dinner?"

print(send_message_gemini(chat, prompt))

Sorry, I can't answer that question. I'm designed to help travelers with their trip planning, and that includes things like finding sights, attractions, and restaurants. I can't help you decide what to eat for dinner, but I can help you find great restaurants in Milan! 

Do you have any preferences for your dinner?  (Type of cuisine, budget, location, etc.) 



You can see that this way, a guardrail in the prompt prevented the chatbot from veering off course.

### Turn generative tasks into classification tasks to reduce output variability

#### Generative tasks lead to higher output variability

The prompt below results in an open-ended response, useful for brainstorming, but response is highly variable.

In [18]:
prompt = "I'm a high school student. Recommend me a programming activity to improve my skills."

print(call_gemini(prompt))

## Programming Activities for High School Students:

Here are some fun and engaging programming activities tailored for high school students, ranging from beginner to more advanced:

**Beginner:**

* **Build a simple text-based game:** Choose a classic game like "Hangman" or "Rock, Paper, Scissors" and code it using a language like Python. This helps understand basic programming concepts like variables, loops, and conditional statements.
* **Create a website with HTML, CSS, and JavaScript:** Design a simple website showcasing your interests or a project you're passionate about. This introduces you to web development fundamentals and provides visual results.
* **Develop a mobile app using MIT App Inventor:** This drag-and-drop interface allows you to create simple Android apps without complex coding. You can learn about app design, user interaction, and basic programming logic.
* **Learn the basics of data science with Python and libraries like NumPy and Pandas:** Explore datasets, anal

#### Classification tasks reduces output variability

The prompt below results in a choice and may be useful if you want the output to be easier to control.

In [19]:
prompt = """I'm a high school student. Which of these activities do you suggest and why:
a) learn Python
b) learn JavaScript
c) learn Fortran
"""

print(call_gemini(prompt))

It's great you're thinking about learning programming! All three languages have their uses, but for a high school student, I would recommend focusing on either **Python** or **JavaScript**. Here's why:

**Python:**

* **Beginner-friendly:** Python is known for its simple syntax and readability, making it easier to learn for beginners.
* **Versatility:**  Python is widely used in various fields like web development, data science, machine learning, and scripting. 
* **Large community and resources:** Python has a huge and active community, providing lots of support, tutorials, and libraries.
* **Excellent for projects:** Python's ease of use allows you to build fun and interesting projects quickly, boosting your motivation.

**JavaScript:**

* **Web development:** JavaScript is essential for building interactive and dynamic websites. It's the language of the web, making it highly valuable.
* **Future-proof:** The demand for JavaScript developers is constantly growing, making it a promisi

### Improve response quality by including examples

Another way to improve response quality is to add examples in your prompt. The LLM learns in-context from the examples on how to respond. Typically, one to five examples (shots) are enough to improve the quality of responses. Including too many examples can cause the model to over-fit the data and reduce the quality of responses.

Similar to classical model training, the quality and distribution of the examples is very important. Pick examples that are representative of the scenarios that you need the model to learn, and keep the distribution of the examples (e.g. number of examples per class in the case of classification) aligned with your actual distribution.

#### Zero-shot prompt

Below is an example of zero-shot prompting, where you don't provide any examples to the LLM within the prompt itself.

In [20]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **Positive** 



#### One-shot prompt

Below is an example of one-shot prompting, where you provide one example to the LLM within the prompt to give some guidance on what type of response you want.

In [21]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **negative** 



#### Few-shot prompt

Below is an example of few-shot prompting, where you provide a few examples to the LLM within the prompt to give some guidance on what type of response you want.

In [22]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment: positive

Tweet: That was awful. Super boring 😠
Sentiment: negative

Tweet: Something surprised me about this video - it was actually original. It was not the same old recycled stuff that I always see. Watch it - you will not regret it.
Sentiment:
"""

print(call_gemini(prompt))

Sentiment: **positive** 

Here's why:

* **"Something surprised me"** implies a positive experience.
* **"It was actually original"** is a strong compliment.
* **"It was not the same old recycled stuff"** further emphasizes the positive aspect of originality.
* **"Watch it - you will not regret it"**  is a clear recommendation, suggesting a positive experience for others. 



#### Choosing between zero-shot, one-shot, few-shot prompting methods

Which prompt technique to use will solely depends on your goal. The zero-shot prompts are more open-ended and can give you creative answers, while one-shot and few-shot prompts teach the model how to behave so you can get more predictable answers that are consistent with the examples provided.